In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold

In [ ]:
# read in data
df = pd.read_csv ('train.csv')
df1 = pd.read_csv('test.csv')

In [ ]:
# Impute missing values using mode
imp_freq = SimpleImputer(strategy="most_frequent")
i = 0
dele_list = []
ave_list = []
while i < 1205:
  name = 'x'+str(i+1)
  if df.iloc[:,i+3].isnull().sum() >= 17398:
    dele_list.append(name)
    i+=1
    continue
  value = df.iloc[:,i+3].values.reshape(-1,1)
  imp = imp_freq.fit_transform(value)
  col = df.iloc[:,i+3].dropna()
  values, counts = np.unique(col, return_counts=True)
  freq_ind = np.argmax(counts)
  ave_list.append(values[freq_ind])
  df.iloc[:,i+3] = imp
  i += 1

print(len(ave_list))
print(len(dele_list))

1199
6


In [ ]:
print(dele_list)

['x96', 'x97', 'x98', 'x1021', 'x1098', 'x1099']


In [ ]:
# data drop deleting columns
df.drop(columns=dele_list, inplace=True)
df1.drop(columns=dele_list, inplace=True)

In [ ]:
# imputation for test data set
i = 0
while i < len(ave_list):
  name = 'x'+str(i+1)
  df1.iloc[:, i+3].fillna(ave_list[i], inplace=True)
  i += 1

In [ ]:
df1["x80"] = df["x80"].map({1:0, 2:0, 4:0, 3:1})
df["x80"] = df["x80"].map({1:0, 2:0, 4:0, 3:1})

In [ ]:
col_cat_list = [16,17,20,21,78,79,81,82,83,84,85,86,87,148,149,150,196,197,
         198,199,200,303,304,305,322,323,324,325,326,327,328,329,330,
         332,434,435,444,472,474,476,480,498,536,537,548,573,574,575,
         576,580,581,582,583,587,588,589,590,634,635,636,637,639,653,
         654,659,660,661,662,663,664,668,677,678,686,690,694,703,707,
         710,754,766,767,780,781,785,787,790,794,804,805,811,837,853,
         867,868,872,874,877,891,892,893,895,896,897,911,915,919,928,
         929,930,944,948,952,965,968,970,971,974,978,980,981,984,985,
         988,990,991,994,995,998,1000,1001,1004,1010,1011,1014,1015,
         1018,1020,1023,1024,1027,1031,1032,1033,1034,1039,1044,1047,
         1051,1054,1058,1061,1065,1068,1072,1075,1079,1082,1088,1110,
         1111,1118,1119,1120,1121,1122,1123,1124,1128,1133,1135,1143,
         1144,1146,1147,1149,1150,1175,1180,1184,1202]
add = [213, 640]
dele = [322, 323, 324, 325, 326, 327, 328, 329, 330, 332, 480, 
        498, 575, 582, 589, 660, 662, 663, 694, 710, 794, 893, 
        995, 1032, 1118, 1119, 1120, 1121, 1122, 1123, 1124, 
        1128, 1143, 1146, 1147, 1149, 1150]
def Diff(li1, li2):
     return list(set(li1) - set(li2))

col_cat_list = Diff(col_cat_list, dele)
col_cat_list.extend(add)
print(col_cat_list)
# 164, 
col_list = []
for i in range(len(col_cat_list)):
  col_list.append('x' + str(col_cat_list[i]))

for col in col_list:
    df[col] = df[col].astype('category',copy=False)
    df1[col] = df1[col].astype('category',copy=False)

[1024, 1027, 1031, 1033, 1034, 1039, 16, 17, 20, 21, 1044, 1047, 536, 537, 1051, 1054, 1058, 548, 1061, 1065, 1068, 1072, 1075, 1079, 1082, 573, 574, 576, 1088, 580, 581, 583, 587, 588, 590, 78, 79, 81, 82, 83, 84, 85, 86, 87, 1110, 1111, 1133, 1135, 1144, 634, 635, 636, 637, 639, 653, 654, 659, 148, 149, 150, 661, 664, 668, 1175, 1180, 1184, 677, 678, 686, 690, 1202, 703, 707, 196, 197, 198, 199, 200, 754, 766, 767, 780, 781, 785, 787, 790, 804, 805, 811, 303, 304, 305, 837, 853, 867, 868, 872, 874, 877, 891, 892, 895, 896, 897, 911, 915, 919, 928, 929, 930, 944, 434, 435, 948, 952, 444, 965, 968, 970, 971, 974, 978, 980, 981, 472, 984, 474, 985, 476, 988, 990, 991, 994, 998, 1000, 1001, 1004, 1010, 1011, 1014, 1015, 1018, 1020, 1023, 213, 640]


In [ ]:
print(len(col_cat_list))

146


In [ ]:
df.dtypes

uniqueid       int64
year           int64
personid       int64
x1           float64
x2           float64
              ...   
x1202       category
x1203        float64
x1204        float64
x1205        float64
health         int64
Length: 1203, dtype: object

In [ ]:
y = df.iloc[:,-1]
x = df.drop(columns=['uniqueid', 'health'])
x['new1'] = np.random.uniform(0,1,17398)
x['new2'] = np.random.randint(0,2,17398)
x['new3'] = np.random.randint(low=0, high=20, size=17398)
#x['personid']= x['personid'].map(str)
x_columns = x.columns.values.tolist()
#print(x_columns)
x

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has po

,year,personid,x1,x2,x3,x4,x5,x6,x7,x8,...,x1199,x1200,x1201,x1202,x1203,x1204,x1205,new1,new2,new3
0,2005,3999,5.0,3.0,5.0,4.0,3.0,2.0,5.0,7.0,...,2.0,1.0,2,160,1.0,4.0,2.0,0.726488,1,12
1,2005,3997,3.0,3.0,3.0,3.0,3.0,2.0,5.0,7.0,...,2.0,1.0,2,140,3.0,5.0,2.0,0.350056,1,6
2,2005,3996,4.0,4.0,3.0,3.0,3.0,2.0,5.0,7.0,...,2.0,1.0,2,140,3.0,5.0,2.0,0.721656,0,0
3,2005,3995,3.0,2.0,3.0,3.0,3.0,2.0,5.0,7.0,...,2.0,1.0,2,150,1.0,5.0,2.0,0.621261,0,12
4,2005,3993,4.0,2.0,4.0,4.0,3.0,2.0,2.0,7.0,...,2.0,1.0,2,130,5.0,8.0,2.0,0.104142,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17393,2017,514,3.0,3.0,3.0,3.0,3.0,2.0,3.0,7.0,...,1.0,1.0,1,60,4.0,9.0,9.0,0.264383,1,18
17394,2017,513,3.0,3.0,4.0,4.0,3.0,2.0,5.0,7.0,...,1.0,1.0,1,80,6.0,8.0,7.0,0.817237,1,16
17395,2017,512,2.0,2.0,2.0,2.0,3.0,2.0,2.0,7.0,...,2.0,1.0,2,150,3.0,3.0,1.0,0.830638,0,1
17396,2017,511,2.0,2.0,4.0,3.0,3.0,2.0,8.0,7.0,...,2.0,2.0,1,90,3.0,4.0,7.0,0.204889,1,6


In [ ]:
clf_f_s = RandomForestClassifier(n_estimators=300, max_features=150, bootstrap=True, oob_score=True)
clf_f_s.fit(x,y)
f = clf_f_s.feature_importances_
f
print(len(f))

1204


In [ ]:
imp_new1 = f[1201]
imp_new2 = f[1202]
imp_new3 = f[1203]
dele_i_list = []
for i in range(len(f)):
  if f[i] <= imp_new1 and f[i] <= imp_new2 and f[i] <= imp_new3:
    dele_i_list.append(i)
print(imp_new1, imp_new2, imp_new3)

0.008196195413027506 0.0010736128837024706 0.00544283547464917


In [ ]:
print(len(dele_i_list))
print(max(dele_i_list))
dele_i_list.remove(max(dele_i_list))
df.drop(df.columns[dele_i_list], axis=1, inplace=True)
df1.drop(df1.columns[dele_i_list], axis=1, inplace=True)

941
1202


In [ ]:
x, y = df.iloc[:,:-1],df.iloc[:,-1]

In [ ]:
x

,year,personid,x1,x2,x3,x6,x13,x16,x19,x110,...,x1178,x1180,x1182,x1183,x1184,x1201,x1202,x1203,x1204,x1205
0,2005,3999,5.0,3.0,5.0,2.0,3.0,1.0,1.0,3.0,...,3.0,3.0,2,41,2,2,160,1.0,4.0,2.0
1,2005,3997,3.0,3.0,3.0,2.0,3.0,1.0,1.0,3.0,...,2.0,3.0,1,49,5,2,140,3.0,5.0,2.0
2,2005,3996,4.0,4.0,3.0,2.0,3.0,1.0,1.0,3.0,...,3.0,3.0,1,49,5,2,140,3.0,5.0,2.0
3,2005,3995,3.0,2.0,3.0,2.0,3.0,1.0,1.0,3.0,...,2.0,3.0,1,74,5,2,150,1.0,5.0,2.0
4,2005,3993,4.0,2.0,4.0,2.0,3.0,1.0,1.0,3.0,...,2.0,3.0,1,57,2,2,130,5.0,8.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17393,2017,514,3.0,3.0,3.0,2.0,3.0,1.0,1.0,3.0,...,2.0,3.0,1,68,3,1,60,4.0,9.0,9.0
17394,2017,513,3.0,3.0,4.0,2.0,3.0,1.0,1.0,3.0,...,2.0,3.0,1,68,3,1,80,6.0,8.0,7.0
17395,2017,512,2.0,2.0,2.0,2.0,3.0,1.0,1.0,3.0,...,2.0,2.0,1,52,5,2,150,3.0,3.0,1.0
17396,2017,511,2.0,2.0,4.0,2.0,3.0,1.0,1.0,3.0,...,2.0,3.0,1,73,2,1,90,3.0,4.0,7.0


# Light Gradient Boosting

In [ ]:
state = 0
test_size = 0.2

In [ ]:
clf_ls = []
kf = KFold(n_splits=5, random_state=42, shuffle=True)
kf.get_n_splits(x)
sum1 = 0
sum2 = 0
sum3 = 0

for train_index, test_index in kf.split(x):
  clf_lgbm = LGBMClassifier()
  x_train, x_test = x.iloc[train_index], x.iloc[test_index]
  y_train, y_test = y[train_index], y[test_index]
  # x_smote_resampled, y_smote_resampled = SMOTE().fit_resample(x_train, y_train)
  # clf_lgbm.fit(x_smote_resampled, y_smote_resampled)
  clf_lgbm.fit(x_train, y_train)
  a = log_loss(y_test, clf_lgbm.predict_proba(x_test))
  b = clf_lgbm.score(x_test, y_test)
  c = clf_lgbm.score(x_train, y_train)
  print(a, b, c)
  sum1 += a
  sum2 += b
  sum3 += c
clf_ls.append((sum1/5, sum2/5, sum3/5))

1.1716869642786794 0.48735632183908045 0.7725966374479092
1.1787693916518274 0.47902298850574715 0.774249173731858
1.1789361979712039 0.4781609195402299 0.7696508118982612
1.1974698596574802 0.4831848232250647 0.7785042028881385
1.1602393413221723 0.49209542972118425 0.7714634672031037


In [ ]:
clf_ls

[(1.1774203509762726, 0.4839640965662613, 0.7732928586338541)]

In [ ]:
lst1 = []

for j in np.arange(10, 61, 10):
  kf = KFold(n_splits=5, random_state=42, shuffle=True)
  kf.get_n_splits(x)
  sum1 = 0
  sum2 = 0
  sum3 = 0

  for train_index, test_index in kf.split(x):
      lgbm = LGBMClassifier(n_estimators=j)
      x_train, x_test = x.iloc[train_index], x.iloc[test_index]
      y_train, y_test = y[train_index], y[test_index]
      # x_smote_resampled, y_smote_resampled = SMOTE().fit_resample(x_train, y_train)
      # lgbm.fit(x_smote_resampled, y_smote_resampled)
      lgbm.fit(x_train, y_train)
      a = log_loss(y_test, lgbm.predict_proba(x_test))
      b = lgbm.score(x_test, y_test)
      c = lgbm.score(x_train, y_train)
      # print(a, b, c)
      sum1 += a
      sum2 += b
      sum3 += c
  lst1.append(('n_estimators={}, loss={}, test_score={}, train_score={}'.format(j, sum1/5, sum2/5, sum3/5)))
  print(lst1[-1])

n_estimators=10, loss=1.2361343999447705, test_score=0.4680434495313424, train_score=0.5327768162813278
n_estimators=20, loss=1.1978083632219345, test_score=0.4799410585020798, train_score=0.5996235285338661
n_estimators=30, loss=1.185714687148543, test_score=0.4802280183564441, train_score=0.651698463843335
n_estimators=40, loss=1.1815415805443656, test_score=0.4801703984167732, train_score=0.6912720003890884
n_estimators=50, loss=1.181012984457532, test_score=0.4809177396067703, train_score=0.7290205414342694
n_estimators=60, loss=1.1813479203212083, test_score=0.4814351792198181, train_score=0.7629181392421931


In [ ]:
lst1 = []
kf = KFold(n_splits=5, random_state=42, shuffle=True)
kf.get_n_splits(x)

for i in [1e-5, 1e-3, 1e-1, 0, 0.1, 0.3, 0.5, 0.7, 0.9, 1]:
  for j in [1e-5, 1e-3, 1e-1, 0, 0.1, 0.3, 0.5, 0.7, 0.9, 1]:
    sum1 = 0
    sum2 = 0
    sum3 = 0

    for train_index, test_index in kf.split(x):
        lgbm = LGBMClassifier(n_estimators=50, learning_rate=0.1, max_depth=9, 
                              num_leaves=20, min_child_samples=100, subsample=1, 
                              colsample_bytree=0.7, reg_alpha=i, reg_lambda=j)
        x_train, x_test = x.iloc[train_index], x.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        # x_smote_resampled, y_smote_resampled = SMOTE().fit_resample(x_train, y_train)
        # lgbm.fit(x_smote_resampled, y_smote_resampled)
        lgbm.fit(x_train, y_train)
        a = log_loss(y_test, lgbm.predict_proba(x_test))
        b = lgbm.score(x_test, y_test)
        c = lgbm.score(x_train, y_train)
        # print(a, b, c)
        sum1 += a
        sum2 += b
        sum3 += c
    lst1.append(('l1={}, l2={}, loss={}, test_score={}, train_score={}'.format(i, j, sum1/5, sum2/5, sum3/5)))
    print(lst1[-1])

l1=1e-05, l2=1e-05, loss=1.1770842904412016, test_score=0.48528644775054264, train_score=0.5743332466708082
l1=1e-05, l2=0.001, loss=1.1769993856049306, test_score=0.4845966769417821, train_score=0.5737297436843373
l1=1e-05, l2=0.1, loss=1.1778713162700813, test_score=0.4830444407000294, train_score=0.5724221433843196
l1=1e-05, l2=0, loss=1.1770842877014367, test_score=0.48528644775054264, train_score=0.5743332466708082
l1=1e-05, l2=0.1, loss=1.1778713162700813, test_score=0.4830444407000294, train_score=0.5724221433843196
l1=1e-05, l2=0.3, loss=1.1770897212380103, test_score=0.486320732275426, train_score=0.5717898376604046
l1=1e-05, l2=0.5, loss=1.1765700473569853, test_score=0.4867230146065225, train_score=0.5731837047348886
l1=1e-05, l2=0.7, loss=1.1777486811569755, test_score=0.4870682386602042, train_score=0.5729107358474302
l1=1e-05, l2=0.9, loss=1.1768797227822183, test_score=0.4879876797732206, train_score=0.573054405747721
l1=1e-05, l2=1, loss=1.177289605216793, test_score=0.

In [ ]:
lst2 = []

for i in np.arange(0.0030, 0.0031, 0.0005):
  for j in np.arange(5, 41, 5):
    kf = KFold(n_splits=5, random_state=42, shuffle=True)
    kf.get_n_splits(x)
    sum1 = 0
    sum2 = 0
    sum3 = 0

    for train_index, test_index in kf.split(x):
        lgbm = LGBMClassifier(n_estimators=350, learning_rate=0.1, max_depth=6, num_leaves=6, 
                              min_child_weight=i, min_child_samples=j)
        x_train, x_test = x.iloc[train_index], x.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        x_smote_resampled, y_smote_resampled = SMOTE().fit_resample(x_train, y_train)
        lgbm.fit(x_smote_resampled, y_smote_resampled)
        a = log_loss(y_test, lgbm.predict_proba(x_test))
        b = lgbm.score(x_test, y_test)
        c = lgbm.score(x_train, y_train)
        # print(a, b, c)
        sum1 += a
        sum2 += b
        sum3 += c
    lst2.append(('min_child_weight={}, min_child_samples={}, loss={}, test_score={}, train_score={}'.format(i, j, sum1/5, sum2/5, sum3/5)))
    print(lst2[-1])

min_child_weight=0.003, min_child_samples=5, loss=1.1788577403565923, test_score=0.47925113901801614, train_score=0.59544197789768
min_child_weight=0.003, min_child_samples=10, loss=1.1774531156054837, test_score=0.48045836595930264, train_score=0.5962035743496173
min_child_weight=0.003, min_child_samples=15, loss=1.1776140720016235, test_score=0.4822977933281131, train_score=0.5951114974831132
min_child_weight=0.003, min_child_samples=20, loss=1.177857998995282, test_score=0.4763198402236076, train_score=0.5972669073250887
min_child_weight=0.003, min_child_samples=25, loss=1.179072225983749, test_score=0.4752278036032286, train_score=0.5963904055117573
min_child_weight=0.003, min_child_samples=30, loss=1.1775995701717448, test_score=0.47810197804230964, train_score=0.5943786955094748
min_child_weight=0.003, min_child_samples=35, loss=1.1770797888901996, test_score=0.47545763910226546, train_score=0.5939475794721044
min_child_weight=0.003, min_child_samples=40, loss=1.1786223090607073,

In [ ]:
lst3 = []
sl = [0.7, 0.8, 0.9, 1.0]
cl = [0.7, 0.8, 0.9, 1.0]

for i in sl:
  for j in cl:
    kf = KFold(n_splits=5, random_state=42, shuffle=True)
    kf.get_n_splits(x)
    sum1 = 0
    sum2 = 0
    sum3 = 0

    for train_index, test_index in kf.split(x):
        lgbm = LGBMClassifier(n_estimators=350, learning_rate=0.1, max_depth=6, num_leaves=6, 
                              min_child_weight=0.0005, min_child_samples=35, subsample=i, colsample_bytree=j)
        x_train, x_test = x.iloc[train_index], x.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        x_smote_resampled, y_smote_resampled = SMOTE().fit_resample(x_train, y_train)
        lgbm.fit(x_smote_resampled, y_smote_resampled)
        a = log_loss(y_test, lgbm.predict_proba(x_test))
        b = lgbm.score(x_test, y_test)
        c = lgbm.score(x_train, y_train)
        # print(a, b, c)
        sum1 += a
        sum2 += b
        sum3 += c
    lst3.append(('subsample={}, colsample_bytree={}, loss={}, test_score={}, train_score={}'.format(i, j, sum1/5, sum2/5, sum3/5)))
    print(lst3[-1])

subsample=0.7, colsample_bytree=0.7, loss=1.1780181201822442, test_score=0.47999852976644763, train_score=0.5918783662018587


In [ ]:
lst4 = []

# for i in np.arange(0.7, 1.1, 0.1):
for j in np.arange(0, 0.5, 0.1):
  kf = KFold(n_splits=5, random_state=42, shuffle=True)
  kf.get_n_splits(x)
  sum1 = 0
  sum2 = 0
  sum3 = 0

  for train_index, test_index in kf.split(x):
      lgbm = LGBMClassifier(n_estimators=350, learning_rate=0.1, max_depth=6, num_leaves=6, 
                            min_child_weight=0.0005, min_child_samples=35)
      x_train, x_test = x.iloc[train_index], x.iloc[test_index]
      y_train, y_test = y[train_index], y[test_index]
      x_smote_resampled, y_smote_resampled = SMOTE().fit_resample(x_train, y_train)
      lgbm.fit(x_smote_resampled, y_smote_resampled)
      a = log_loss(y_test, lgbm.predict_proba(x_test))
      b = lgbm.score(x_test, y_test)
      c = lgbm.score(x_train, y_train)
      # print(a, b, c)
      sum1 += a
      sum2 += b
      sum3 += c
  lst4.append(('L1={}, loss={}, test_score={}, train_score={}'.format(j, sum1/5, sum2/5, sum3/5)))
  print(lst4[-1])

In [ ]:
x

In [ ]:
x_smote_resampled1, y_smote_resampled1 = SMOTE().fit_resample(x, y)

In [ ]:
lst1 = []
for i in np.arange(300, 410, 10):
      print("depth = ", i)
      sum1 = 0
      sum2 = 0
      sum3 = 0
      for k in range(1, 6):
        lgbm = LGBMClassifier()
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=5 * k)
        x_smote_resampled, y_smote_resampled = SMOTE().fit_resample(x_train, y_train)
        lgbm.fit(x_smote_resampled, y_smote_resampled)
        a = log_loss(y_test, lgbm.predict_proba(x_test))
        b = lgbm.score(x_test,y_test)
        c = lgbm.score(x_train,y_train)
        print(a, b, c)
        sum1 += a
        sum2 += b
        sum3 += c
      lst1.append((i, sum3/5, sum2/5, sum1/5))
      print(i," ", "score_train ", sum3/5, "score_test ", sum2/5, "loss ", sum1/5)

depth =  300
1.1816957406188682 0.47988505747126436 0.6965799683862623
1.1999709167898063 0.4704022988505747 0.7003879867797097
1.1892444634321586 0.47155172413793106 0.7044834027877569
1.1714409109419452 0.48074712643678164 0.6965799683862623
1.2118905153856503 0.45488505747126434 0.6983762034775112
300   score_train  0.6992815059635005 score_test  0.47149425287356317 loss  1.1908485094336858
depth =  310
1.1809395468172739 0.4732758620689655 0.6937778416439143
1.200981353941164 0.4752873563218391 0.6980169564592614


In [ ]:
params={'n_estimators':np.arange(50, 600, 50)}

clf = LGBMClassifier()
grid = GridSearchCV(clf, params, cv=5, scoring="neg_log_loss", verbose=3)
grid.fit(x_smote_resampled1, y_smote_resampled1)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5] END ....reg_alpha=0.1, reg_lambda=0.1;, score=-1.873 total time= 1.4min
[CV 2/5] END ....reg_alpha=0.1, reg_lambda=0.1;, score=-1.635 total time= 1.3min
[CV 3/5] END ....reg_alpha=0.1, reg_lambda=0.1;, score=-1.750 total time= 1.4min
[CV 4/5] END ....reg_alpha=0.1, reg_lambda=0.1;, score=-1.234 total time= 1.3min
[CV 5/5] END ....reg_alpha=0.1, reg_lambda=0.1;, score=-1.737 total time= 1.3min


KeyboardInterrupt: ignored

In [ ]:
a = log_loss(y_test, lgbm.predict_proba(x_test))
b = lgbm.score(x_test,y_test)
c = lgbm.score(x_train,y_train)
print(a, b, c)

1.1762527731049344 0.4761494252873563 0.5954878574507831


In [ ]:
def getloss(x):
  return x[4]

lst.sort(key=getloss)

In [ ]:
lst

[(6, 8, 0.592886909038655, 0.4813218390804598, 1.1759208512624064),
 (7, 10, 0.6246443454519327, 0.48178160919540225, 1.1762469758322576),
 (7, 8, 0.5935335536715045, 0.4787356321839081, 1.1765190119704063),
 (8, 8, 0.592786319873545, 0.48293103448275854, 1.176619052722192),
 (7, 12, 0.6574651530392297, 0.47793103448275864, 1.1776833275223224),
 (5, 8, 0.5901853714614169, 0.47988505747126436, 1.1777031073434425),
 (8, 10, 0.6260382238827418, 0.48166666666666663, 1.1779081836390375),
 (6, 12, 0.6568616180485702, 0.4780459770114942, 1.1784719145911111),
 (8, 12, 0.6583417157637592, 0.4789655172413793, 1.1785083280552167),
 (5, 10, 0.6221296163241845, 0.48091954022988503, 1.1788359522583838),
 (6, 10, 0.6247449346170427, 0.4772413793103449, 1.1793744038092502),
 (5, 12, 0.6526943526368731, 0.4801724137931034, 1.180569453690227),
 (7, 14, 0.6877712314987786, 0.48166666666666663, 1.1810030640877518),
 (8, 14, 0.6903003305072568, 0.4791954022988506, 1.1811871838571975),
 (8, 16, 0.7193849691

In [ ]:
print(x.dtypes)

year          int64
personid      int64
x1          float64
x2          float64
x3          float64
             ...   
x1201         int64
x1202         int64
x1203       float64
x1204       float64
x1205       float64
Length: 287, dtype: object


In [ ]:
lst1 = []
for i in np.arange(5, 8):
  for j in np.arange(7, 10, 1):
    print("max_depth = ", i, "num_leaves = ", j)
    sum1 = 0
    sum2 = 0
    sum3 = 0
    for k in range(1, 6):
      lgbm = LGBMClassifier(n_estimators=500, learning_rate=0.05, max_depth=i, num_leaves=j,
          colsample_bytree = 0.8, subsample = 0.8, lambda_l1= 0.1,
          lambda_l2=0.2)
      x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=k)
      x_smote_resampled, y_smote_resampled = SMOTE().fit_resample(x_train, y_train)
      lgbm.fit(x_smote_resampled, y_smote_resampled)
      a = log_loss(y_test, lgbm.predict_proba(x_test))
      b = lgbm.score(x_test,y_test)
      c = lgbm.score(x_train,y_train)
      print(a, b, c)
      sum1 += a
      sum2 += b
      sum3 += c
    lst1.append((i, j, sum3/5, sum2/5, sum1/5))
    print(i," ", j, "score_train ", sum3/5, "score_test ", sum2/5, "loss ", sum1/5)

max_depth =  5 num_leaves =  7
1.1798588033859734 0.4850574712643678 0.5747233797959477
1.1839258883812152 0.4755747126436782 0.5767351630981463
1.164562112132989 0.4810344827586207 0.5729989941083489
1.1797660087954605 0.4853448275862069 0.5755855726397471
1.1808109293727702 0.4818965517241379 0.5778129041528955
5   7 score_train  0.5755712027590171 score_test  0.48178160919540236 loss  1.1777847484136816
max_depth =  5 num_leaves =  8
1.178209060324411 0.47586206896551725 0.591320592039086
1.1863114191162654 0.4752873563218391 0.5906739474062366
1.164302239858156 0.48735632183908045 0.5939071705704843
1.1777455599684377 0.4885057471264368 0.5935479235522345
1.1758018837517723 0.4810344827586207 0.5922546342865355
5   8 score_train  0.5923408535709153 score_test  0.48160919540229885 loss  1.1764740326038083
max_depth =  5 num_leaves =  9
1.1818874415391345 0.47701149425287354 0.6069119126311252
1.183848612399002 0.4721264367816092 0.6083489007041242
1.163307309306151 0.484770114942528

In [ ]:
x, y = df.iloc[:,:-1],df.iloc[:,-1]
lgbm1 = LGBMClassifier(n_estimators=500, learning_rate=0.05, max_depth=5, num_leaves=7,
          colsample_bytree = 0.8, subsample = 0.8, lambda_l1= 0.1,
          lambda_l2=0.2)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size, random_state=42)
x_smote_resampled, y_smote_resampled = SMOTE().fit_resample(x_train, y_train)
lgbm1.fit(x_smote_resampled, y_smote_resampled)

LGBMClassifier(colsample_bytree=0.8, lambda_l1=0.1, lambda_l2=0.2,
               learning_rate=0.05, max_depth=5, n_estimators=500, num_leaves=7,
               subsample=0.8)

##### Gradient Boosting prediction

In [ ]:
x, y = df.iloc[:,:-1],df.iloc[:,-1]
gb = GradientBoostingClassifier(n_estimators=90)
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
gb.fit(x, y)

In [ ]:
df2 = pd.read_csv("test.csv")

In [ ]:
pred = gb.predict_proba(df1).T
pred = pd.DataFrame(pred)
pred = pred.append(df2['uniqueid'])
pred = pred.T
pred.columns = ['p1', 'p2', 'p3', 'p4', 'p5', 'uniqueid']
pred = pred[['uniqueid', 'p1', 'p2', 'p3', 'p4', 'p5']]
pred['uniqueid'] = pred['uniqueid'].astype(int)
pred.to_csv('result_lgb_6_8_08_08_005.csv', index=False)